load env 

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["PINECODE_API_KEY"]=os.getenv("PINECODE_API_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"]=os.getenv("HUGGINGFACE_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]=os.getenv("LANGCHAIN_TRACING_V2")

fetch the data from pdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
# Load PDF document
pdf_loader = PyPDFLoader(
    file_path = "/Users/midas-ai/Documents/Personal/agentic-ai-2.0-course/workspace/2-Langchain- Basics/Assignment/attention-all-you-need.pdf"
)

In [7]:
pages=pdf_loader.load()
len(pages)

15

chunking

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a semantic chunker
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

# Split documents semantically
chunked_docs = text_splitter.split_documents(pages)
len(chunked_docs)

93

embeddings

In [14]:
from langchain.embeddings import OpenAIEmbeddings

embeddings=OpenAIEmbeddings( model="text-embedding-3-large")

len(embeddings.embed_query("hello"))

/var/folders/r6/94rtxyj11sgb8st7ytmz40ph0000gn/T/ipykernel_85897/1523018376.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings( model="text-embedding-3-large")


3072

saving documents in vector store [milvus-local]

In [17]:
from langchain_milvus import Milvus

URI = "./milvus_local.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri":URI},
    index_params={"index_type":"FLAT", "metric_type":"L2"}
)

2025-06-07 20:33:18,519 [DEBUG][_create_connection]: Created new connection using: 8123d3be1ac1458bacd2c2eebfb5ca36 (async_milvus_client.py:599)


In [23]:
from uuid import uuid4
#universal indentification number
uuids = [str(uuid4()) for _ in range(len(chunked_docs))]

In [29]:
import re

def sanitize_metadata_keys(doc):
    new_metadata = {}
    for key, value in doc.metadata.items():
        # Replace invalid characters with underscore
        sanitized_key = re.sub(r'\W|^(?=\d)', '_', key)
        new_metadata[sanitized_key] = value
    doc.metadata = new_metadata
    return doc

# Apply to all documents
chunked_docs = [sanitize_metadata_keys(doc) for doc in chunked_docs]

vector_store.add_documents(chunked_docs)

[458570899064094720,
 458570899064094721,
 458570899064094722,
 458570899064094723,
 458570899064094724,
 458570899064094725,
 458570899064094726,
 458570899064094727,
 458570899064094728,
 458570899064094729,
 458570899064094730,
 458570899064094731,
 458570899064094732,
 458570899064094733,
 458570899064094734,
 458570899064094735,
 458570899064094736,
 458570899064094737,
 458570899064094738,
 458570899064094739,
 458570899064094740,
 458570899064094741,
 458570899064094742,
 458570899064094743,
 458570899064094744,
 458570899064094745,
 458570899064094746,
 458570899064094747,
 458570899064094748,
 458570899064094749,
 458570899064094750,
 458570899064094751,
 458570899064094752,
 458570899064094753,
 458570899064094754,
 458570899064094755,
 458570899064094756,
 458570899064094757,
 458570899064094758,
 458570899064094759,
 458570899064094760,
 458570899064094761,
 458570899064094762,
 458570899064094763,
 458570899064094764,
 458570899064094765,
 458570899064094766,
 458570899064

In [35]:
import time
# Start the timer
start_time = time.time()
# Operation to measure
retriever=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1} #hyperparameter
)
results=retriever.invoke("self-attention mechanism")
# End the timer
end_time = time.time()
# Print duration
print(f"Time taken to retrieve documents: {end_time - start_time:.2f} seconds")

for result in results:
    print(result.page_content)

Time taken to retrieve documents: 0.90 seconds
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
• In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].
• The encoder contains self-attention layers. In a self-attention layer all of the keys, values
(x1, ..., xn) to another sequence of equal length (z1, ..., zn), wit

In [40]:
query = "self-attention mechanism"
results_with_scores = vector_store.similarity_search_with_score(query)

for i, (doc, score) in enumerate(results_with_scores):
    print(f"Result {i+1}:")
    print(f"Document Content: {doc.page_content}")
    print(f"Similarity Score: {score:.4f}\n")

Result 1:
Document Content: reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
Similarity Score: 0.6449

Result 2:
Document Content: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].
• The encoder contains self-attention layers. In a self-attention layer all of the keys, values
Similarity Score: 0.7812

Result 

In [45]:
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever
from langchain.schema import Document

# Step 1: Vector similarity search
query = "self-attention mechanism"
top_k = 3
initial_results = vector_store.similarity_search(query, k=top_k)

for i, doc in enumerate(initial_results):
    print(f"Initial {i+1}:")
    print(doc.page_content[:20])
    print()

# Step 2: Prepare documents for BM25
bm25_retriever = BM25Retriever.from_documents(initial_results)

# Step 3: Perform reranking using BM25
reranked_docs = bm25_retriever.get_relevant_documents(query)

# Step 4: Print results
for i, doc in enumerate(reranked_docs):
    print(f"Rank {i+1}:")
    print(doc.page_content[:20])
    print()

Initial 1:
reduced to a constan

Initial 2:
• In "encoder-decode

Initial 3:
(x1, ..., xn) to ano

Rank 1:
reduced to a constan

Rank 2:
• In "encoder-decode

Rank 3:
(x1, ..., xn) to ano



In [46]:
from langchain import hub
prompt=hub.pull("rlm/rag-prompt")

In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    { "context": retriever | format_docs , "question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
    )

In [50]:
response= rag_chain.invoke("What is Transformers?")

saving the llm output in docx

In [55]:

# Step 3: Write the LLM output to a new .docx file
output_doc = Document()
output_doc.add_heading("LLM Output", level=1)
output_doc.add_paragraph(response)
output_doc.save("./output.docx")

TypeError: Document.__init__() missing 1 required positional argument: 'page_content'